In [3]:
#血統のデータをスクレイプする 特徴量を作成する 当日も取れる情報がよい
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from tqdm import tqdm as tqdm
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 100)

In [4]:
def sclape_peds(horse_id_list, pre_peds={}):
    peds = pre_peds
    horse_top = {}
    for horse_id in tqdm(horse_id_list):
        if horse_id in peds.keys():
            continue
        try:
            url = 'https://db.netkeiba.com/horse/ped/' + horse_id
            df = pd.read_html(url)[0]
            generations = {}
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()

            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
            
            
            url2 = 'https://db.netkeiba.com/horse/' + horse_id
            horse_top[horse_id] = pd.read_html(url2)[1].set_index(0).T.reset_index(drop=True)
            time.sleep(0.1)
            
        except IndexError:
            print("IndexError")
            continue
        except Exception as e:
            print(e)
            # for文から抜け出す
            print("bleak")
            break
            
    peds_df = pd.concat([peds[horse_id] for horse_id in peds], axis=1).T.add_prefix('peds_')
    
    for key in horse_top.keys():
        horse_top[key].index = [key] * len(horse_top[key])

    horse_top_df = pd.concat([horse_top[key] for key in horse_top.keys()], sort=False)
    
    
    peds_final = peds_df.merge(horse_top_df, left_index=True, right_index=True, how='left')

    return peds_final

In [5]:
def sclape_peds_dic(horse_id_list, pre_peds={}):
    peds = pre_peds
    horse_top = {}
    for horse_id in tqdm(horse_id_list):
        if horse_id in peds.keys():
            continue
        try:
            url = 'https://db.netkeiba.com/horse/ped/' + horse_id
            df = pd.read_html(url)[0]
            generations = {}
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()

            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
            
            
            url2 = 'https://db.netkeiba.com/horse/' + horse_id
            horse_top[horse_id] = pd.read_html(url2)[1].set_index(0).T.reset_index(drop=True)
            time.sleep(0.1)
            
        except IndexError:
            print("IndexError")
            continue
        except Exception as e:
            print(e)
            # for文から抜け出す
            print("bleak")
            break
            
    
    return peds, horse_top

In [136]:
peds_df = pd.concat([peds[horse_id] for horse_id in peds], axis=1).T.add_prefix('peds_')
    
for key in horse_top.keys():
    horse_top[key].index = [key] * len(horse_top[key])

horse_top_df = pd.concat([horse_top[key] for key in horse_top.keys()], sort=False)
    
    
peds_final = peds_df.merge(horse_top_df, left_index=True, right_index=True, how='left')

In [6]:
horse_id_list = ['2017105317', '2017101429', '2017100464']
test, test2 = sclape_peds_dic(horse_id_list)
test2

100%|██████████| 3/3 [00:01<00:00,  1.56it/s]


{'2017105317': 0        生年月日        調教師    馬主       生産者   産地                     セリ取引価格  \
 0  2017年1月23日  池江泰寿 (栗東)  吉田勝己  ノーザンファーム  安平町  1億8,360万円 (2017年 セレクトセール)   
 
 0          獲得賞金            通算成績     主な勝鞍                  近親馬  
 0  4,473万円 (中央)  5戦1勝 [1-2-1-1]  19'２歳新馬  トレデマンド、コンドコマンドの2019  ,
 '2017101429': 0        生年月日        調教師   馬主           募集情報         生産者   産地 セリ取引価格  \
 0  2017年4月16日  鹿戸雄一 (美浦)  ウイン  1口:6.5万円/400口  コスモヴューファーム  新冠町      -   
 
 0          獲得賞金             通算成績        主な勝鞍              近親馬  
 0  3,963万円 (中央)  10戦2勝 [2-2-0-6]  20'3歳1勝クラス  ウインルチル、ウインガーネット  ,
 '2017100464': 0       生年月日       調教師   馬主   生産者   産地 セリ取引価格        獲得賞金            通算成績  \
 0  2017年5月9日  森秀行 (栗東)  平井裕  白井牧場  日高町      -  717万円 (中央)  6戦1勝 [1-0-1-4]   
 
 0      主な勝鞍             近親馬  
 0  20'3歳未勝利  オーロール、ヒーラの2019  }

In [7]:
test

{'2017105317': 0                      ディープインパクト 2002 鹿毛 [血統][産駒] Halo系
 1     コンドコマンド Condo Commando(米) 2012 青鹿毛 [血統][産駒] FN...
 2         サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒]
 3       ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒]
 4                        Tiz Wonderful 2004 鹿毛 [血統][産駒]
                             ...                        
 57                                   Summertime Promise
 58                                             Riverman
 59                                      Six Months Long
 60                                         Slew o' Gold
 61                                           Fiscal Fun
 Name: 2017105317, Length: 62, dtype: object,
 '2017101429': 0       スクリーンヒーロー 2004 栗毛 [血統][産駒] Roberto系
 1     コスモクリスタル 2006 黒鹿毛 [血統][産駒] FNo.[13-e]
 2                  グラスワンダー 1995 栗毛 [血統][産駒]
 3                ランニングヒロイン 1993 鹿毛 [血統][産駒]
 4                  マイネルラヴ 1995 青鹿毛 [血統][産駒]
                       ...                  
 57                          

In [7]:
race_results = pd.read_pickle('1-2results2015_2020.pickle')
horse_results = pd.read_pickle('1-4horse_results2015_2020.pickle')

In [130]:
horse_id_list = race_results['horse_id'].unique()

In [131]:
peds, horse_top = sclape_peds_dic(horse_id_list)

100%|██████████| 30095/30095 [5:47:00<00:00,  1.45it/s]  


In [138]:
peds_final.tail()

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,peds_11,peds_12,peds_13,peds_14,peds_15,peds_16,peds_17,peds_18,peds_19,peds_20,peds_21,peds_22,peds_23,peds_24,peds_25,peds_26,peds_27,peds_28,peds_29,peds_30,peds_31,peds_32,peds_33,peds_34,peds_35,peds_36,peds_37,peds_38,peds_39,peds_40,peds_41,peds_42,peds_43,peds_44,peds_45,peds_46,peds_47,peds_48,peds_49,peds_50,peds_51,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61,生年月日,調教師,馬主,生産者,産地,セリ取引価格,獲得賞金,通算成績,主な勝鞍,近親馬,募集情報
2017110035,Dubawi 2002 鹿毛 [血統][産駒] Mr. Prospector系,ミッドサマーフェア 2009 青鹿毛 [血統][産駒] FNo.[1-o],Dubai Millennium 1996 鹿毛 [血統][産駒],Zomaradah 1995 [血統][産駒],タニノギムレット 1999 鹿毛 [血統][産駒],ストロベリーフェア Strawberry Fair(英) 2001 青鹿毛 [血統][産駒],Seeking the Gold 1985 鹿毛 [血統][産駒],Colorado Dancer 1986 [血統][産駒],Deploy 1987 [血統][産駒],Jawaher 1989 [血統][産駒],ブライアンズタイム Brian's Time(米) 1985 黒鹿毛 [血統][産駒],タニノクリスタル 1988 栗毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],ストームソング Storm Song(米) 1994 鹿毛 [血統][産駒],Mr. Prospector 1970 鹿毛,Con Game 1974,Shareef Dancer 1980,Fall Aspen 1976,Shirley Heights 1975 鹿毛,Slightly Dangerous 1979,ダンシングブレーヴ 1983 鹿毛,High Tern 1982,Roberto 1969 鹿毛,Kelley's Day 1977,クリスタルパレス 1974 芦毛,タニノシーバード 1972 栗毛,Mr. Prospector 1970 鹿毛,Miesque 1984 鹿毛,Summer Squall 1987,Hum Along 1989,Raise a Native,Gold Digger,Buckpasser,Broadway,Northern Dancer,Sweet Alliance,Pretense,Change Water,Mill Reef,Hardiemma,Roberto,Where You Lead,Lyphard,Navajo Princess,High Line,Sunbittern,Hail to Reason,Bramalea,Graustark,Golden Trail,Caro,Hermieres,Sea-Bird,Flaxen,Raise a Native,Gold Digger,Nureyev,Pasadoble,Storm Bird,Weekend Surprise,Fappiano,Minstress,2017年2月20日,大久保龍 (栗東),ゴドルフィン,Godolphin,英,-,"2,801万円 (中央)",10戦2勝 [2-3-1-4],20'石狩特別(1勝クラス),ミッドサマーハウス、サマートゥリスト,NaN
2018102033,キズナ 2010 青鹿毛 [血統][産駒] Halo系,ヴァイセフラウ 2011 芦毛 [血統][産駒] FNo.[8-h],ディープインパクト 2002 鹿毛 [血統][産駒],キャットクイル Catequil(加) 1990 鹿毛 [血統][産駒],キングカメハメハ 2001 鹿毛 [血統][産駒],ラティール 1995 芦毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],Storm Cat (米) 1983 [血統][産駒],Pacific Princess 1973 鹿毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],タマモクロス 1984 芦毛 [血統][産駒],ソルティーレディー 1987 黒鹿毛 [血統][産駒],Halo 1969,Wishing Well 1975,Alzao 1980 鹿毛,Burghclere 1977,Storm Bird 1978 鹿毛,Terlingua 1976,Damascus 1964,Fiji 1960,Mr. Prospector 1970 鹿毛,Miesque 1984 鹿毛,ラストタイクーン 1983 黒鹿毛,Pilot Bird 1983,シービークロス 1975 芦毛,グリーンシャトー 1974 栗毛,ニホンピロウイナー 1980 黒鹿毛,ダイナバーディ 1980 黒鹿毛,Hail to Reason,Cosmah,Understanding,Mountain Flower,Lyphard,Lady Rebecca,Busted,Highclere,Northern Dancer,South Ocean,Secretariat,Crimson Saint,Sword Dancer,Kerala,Acropolis,Rififi,Raise a Native,Gold Digger,Nureyev,Pasadoble,トライマイベスト,Mill Princess,Blakeney,The Dancer,フォルティノ,ズイシヨウ,シャトーゲイ,クインビー,スティールハート,ニホンピロエバート,ノーザンテースト,タミーズターン,2018年4月10日,松永幹夫 (栗東),ノースヒルズ,ノースヒルズ,新冠町,-,510万円 (中央),3戦1勝 [1-0-0-2],20'２歳未勝利,シェーン、コントディヴェール,NaN
2018103923,バゴ Bago(仏) 2001 黒鹿毛 [血統][産駒] Blushing Groom系,ドリームバラード 2009 黒鹿毛 [血統][産駒] FNo.[3-n],Nashwan 1986 [血統][産駒],Moonlight's Box 1996 鹿毛 [血統][産駒],タニノギムレット 1999 鹿毛 [血統][産駒],フジャブ Hujjab(米) 1992 鹿毛 [血統][産駒],Blushing Groom (仏) 1974 栗毛 [血統][産駒],Height of Fashion 1979 [血統][産駒],Nureyev 1977 鹿毛 [血統][産駒],Coup de Genie 1991 鹿毛 [血統][産駒],ブライアンズタイム Brian's Time(米) 1985 黒鹿毛 [血統][産駒],タニノクリスタル 1988 栗毛 [血統][産駒],Woodman 1983 [血統][産駒],Winters' Love 1982 [血統][産駒],Red God 1954 栗毛,Runaway Bride 1962,Bustino 1971 鹿毛,Highclere 1971,Northern Dancer 1961 鹿毛,Special 1969,Mr. Prospector 1970 鹿毛,Coup de Folie 1982,Roberto 1969 鹿毛,Kelley's Day 1977,クリスタルパレス 1974 芦毛,タニノシーバード 1972 栗毛,Mr. Prospector 1970 鹿毛,プレイメイト 1975 栗毛,Danzig 1977 鹿毛,Cold Hearted 1974,Nasrullah,Spring Run,Wild Risk,Aimee,Busted,Ship Yard,Queen's Hussar,Highlight,Nearctic,Natalma,Forli,Thong,Raise a Native,Gold Digger,Halo,Raise the Standard,Hail to Reason,Bramalea,Graustark,Golden Trail,Caro,Hermieres,Sea-Bird,Flaxen,Raise a Native,Gold Digger,Buckpasser,Intriguing,Northern Dancer,Pas de Nom,The Axe,Turn to North,2018年2月25日,吉岡辰弥 (栗

In [140]:
peds_final.to_pickle('1-11peds2015_2020.pickle')